## Table of Contents

* [Getting Started - Load Libraries and Dataset](#0)
    * [2.1. Load Libraries](#0.1)    
* [Symbol Search](#1)
* [Historical Price and Volume for 1 Stock](#2)
    * [Adding Time Periods](#2.1)
* [Frequency Setting](#3)
* [Dividends](#4)
* [Financial Indices](#5)
* [Currency Exchange](#6)
* [Cryptocurrencies](#7)
* [Mutual Funds](#8)
* [Treasury Rates](#9)
* [Stock Fundamentals](#10)
* [Financials](#11)
* [Stream Realtime Data](#12)
* [Economic Indicators](#13)
* [Technical Indicators](#14)

<a id='0'></a>
## Getting Started - Load Libraries and Dataset

link: https://github.com/RomelTorres/alpha_vantage

In [ ]:
https://medium.com/codex/alpha-vantage-an-introduction-to-a-highly-efficient-free-stock-api-6d17f4481bf

### Load Libraries

In [1]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import time
import requests
from io import BytesIO


In [2]:
# API Key
key = "AAUUBAYEDCI353AC"

<a id='1'></a>
## Symbol Search

In [18]:
symbol_to_search = 'TSL'
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords='+symbol_to_search+'&apikey={key}'
r = requests.get(url)
data = r.json()
data = pd.DataFrame(data['bestMatches'])


,1. symbol,2. name,3. type,4. region,5. marketOpen,6. marketClose,7. timezone,8. currency,9. matchScore
0,TSLA,Tesla Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.8571
1,TSL.LON,ThinkSmart Limited,Equity,United Kingdom,08:00,16:30,UTC+00,GBX,0.7500
2,TSLAX,TRANSAMERICA SMALL CAP VALUE A,Mutual Fund,United States,09:30,16:00,UTC-04,USD,0.7500
3,TSLCX,Transamerica Small Cap Value Class C,Mutual Fund,United States,09:30,16:00,UTC-04,USD,0.7500
4,TSLFX,TRANSAMERICA SMALL CAP VALUE R4,Mutual Fund,United States,09:30,16:00,UTC-04,USD,0.7500
5,TSLIX,TRANSAMERICA SMALL CAP VALUE I,Mutual Fund,United States,09:30,16:00,UTC-04,USD,0.7500
6,TSLLF,Tassal Group,Equity,United States,09:30,16:00,UTC-04,USD,0.7500
7,TSLMF,ThinkSmart Ltd,Equity,United States,09:30,16:00,UTC-04,USD,0.7500
8,TSL.TRT,Tree Island Steel Ltd,Equity,Toronto,09:30,16:00,UTC-05,CAD,0.6667
9,TSLA34.SAO,Tesla Inc,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,0.5000


<a id='2'></a>
## Historic Stock Price and Volume

In [37]:
data = {
    "function": "DIGITAL_CURRENCY_DAILY", # WEEKLY, MONTHLY possible
    "symbol": "ETH",
    "market": 'CNY',
    "apikey": key
    }
r = requests.get(url, params=data)
data = r.json()
crypto_df = pd.DataFrame(data['Time Series (Digital Currency Daily)']).T.reset_index()
crypto_df = crypto_df.rename(columns={"index": "Date"})
crypto_df['Date'] = pd.to_datetime(crypto_df['Date'])
crypto_df 

,open,high,low,close,volume
date,,,,,
2010-06-29,19.00,25.0000,17.5400,23.89,18766300.0
2010-06-30,25.79,30.4192,23.3000,23.83,17187100.0
2010-07-01,25.00,25.9200,20.2700,21.96,8218800.0
2010-07-02,23.00,23.1000,18.7100,19.20,5139800.0
2010-07-06,20.00,20.0000,15.8300,16.11,6866900.0
...,...,...,...,...,...
2022-01-04,1189.55,1208.0000,1123.0500,1149.59,33416086.0
2022-01-05,1146.65,1170.3400,1081.0101,1088.12,26706599.0
2022-01-06,1077.00,1088.0000,1020.5000,1064.70,30112158.0


<a id='2.1'></a>
## Adding Time Periods

In [35]:
data = {
    "function": "TIME_SERIES_DAILY", # WEEKLY, MONTHLY possible
    "symbol": "TSLA",
     "apikey": key
    }
r = requests.get(url, params=data)
data = r.json()
data = pd.DataFrame(data['Time Series (Daily)']).T
data

,1. open,2. high,3. low,4. close,5. volume
2022-01-10,1000.0000,1059.1000,980.0000,1058.1200,30604959
2022-01-07,1080.3700,1080.9299,1010.0000,1026.9600,28054916
2022-01-06,1077.0000,1088.0000,1020.5000,1064.7000,30112158
2022-01-05,1146.6500,1170.3400,1081.0101,1088.1200,26706599
2022-01-04,1189.5500,1208.0000,1123.0500,1149.5900,33416086
...,...,...,...,...,...
2021-08-25,707.0300,716.9700,704.0000,711.2000,12645562
2021-08-24,710.6800,715.2200,702.6400,708.4900,13083071
2021-08-23,685.4420,712.1299,680.7505,706.3000,20264859
2021-08-20,682.8500,692.1299,673.7000,680.2600,14841865


<a id='3'></a>
## Frequency Setting

In [8]:
def get_intraday_data(symbol, interval):
    api_key = key
    api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df[f'Time Series ({interval})']).T
    df = df.rename(columns = {'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1]
    return df

tsla_intra = get_intraday_data('TSLA', '1min')
tsla_intra.tail()

,open,high,low,close,volume
2021-12-23 19:56:00,1068.45,1068.90,1068.45,1068.5,2502.0
2021-12-23 19:57:00,1068.70,1068.70,1068.70,1068.7,855.0
2021-12-23 19:58:00,1069.00,1069.50,1069.00,1069.5,3332.0
2021-12-23 19:59:00,1069.20,1069.72,1069.05,1069.5,2642.0
2021-12-23 20:00:00,1069.00,1069.30,1069.00,1069.3,4489.0


<a id='4'></a>
## Dividends

In [27]:
ticker = "IBM"
url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+ticker+'&apikey={key}'
r = requests.get(url)
data = r.json()
dividends = pd.DataFrame(data, index = ['Values'])
dividends = dividends[['DividendPerShare', 'DividendYield', 'DividendDate', 'ExDividendDate']].T
data


{'Symbol': 'IBM',
 'AssetType': 'Common Stock',
 'Name': 'International Business Machines Corporation',
 'Description': 'International Business Machines Corporation (IBM) is an American multinational technology company headquartered in Armonk, New York, with operations in over 170 countries. The company began in 1911, founded in Endicott, New York, as the Computing-Tabulating-Recording Company (CTR) and was renamed International Business Machines in 1924. IBM is incorporated in New York. IBM produces and sells computer hardware, middleware and software, and provides hosting and consulting services in areas ranging from mainframe computers to nanotechnology. IBM is also a major research organization, holding the record for most annual U.S. patents generated by a business (as of 2020) for 28 consecutive years. Inventions by IBM include the automated teller machine (ATM), the floppy disk, the hard disk drive, the magnetic stripe card, the relational database, the SQL programming language,

### Earnings Information

In [14]:
BASE_URL = r"https://www.alphavantage.co/query?"

def earnings_calendar_api(api_key, horizon, symbol=None):
    if symbol is not None:
        url = f'{BASE_URL}function=EARNINGS_CALENDAR&symbol={symbol}&horizon={horizon}&apikey={api_key}'
        response = requests.get(url)
    else:
        url = f"{BASE_URL}function=EARNINGS_CALENDAR&horizon={horizon}&apikey={api_key}"
        response = requests.get(url)

    return pd.read_csv(BytesIO(response.content))

api_key = key
horizon=["3month","6month","12month"]
earnings_calendar_api(api_key,horizon[0])

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
0,A,Agilent Technologies Inc,2022-02-14,2022-01-31,1.17,USD
1,AA,Alcoa Corp,2022-01-18,2021-12-31,1.77,USD
2,AADI,Aadi Bioscience Inc,2022-03-09,2021-12-31,-1.06,USD
3,AAIC,Arlington Asset Investment Corp - Class A,2022-02-14,2021-12-31,0.06,USD
4,AAL,American Airlines Group Inc,2022-01-26,2021-12-31,-1.79,USD
...,...,...,...,...,...,...
4788,ZUO,Zuora Inc - Class A,2022-03-09,2022-01-31,-0.02,USD
4789,ZVO,Zovio Inc,2022-02-22,2021-12-31,-0.11,USD
4790,ZYME,Zymeworks Inc,2022-02-22,2021-12-31,-1.18,USD
4791,ZYNE,Zynerba Pharmaceuticals Inc,2022-03-08,2021-12-31,-0.26,USD


<a id='5'></a>
## Financial Indices

In [23]:
index = "DJI" # FCHI, IXIC, ...
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+index+'&outputsize=full&apikey={key}'
r = requests.get(url)
data = r.json()


{'Global Quote': {}}

<a id='6'></a>
## Currency Exchange

In [25]:
# Currency list: https://www.alphavantage.co/physical_currency_list/
currency_a = 'EUR'
currency_b = 'USD'
interval = '5min' # 1min, 5min, 15min, 30min, 60min
url = 'https://www.alphavantage.co/query?function=FX_INTRADAY&from_symbol=EUR&to_symbol=USD&interval=5min&apikey=demo'
r = requests.get(url)
data = r.json()


{'Meta Data': {'1. Information': 'FX Intraday (5min) Time Series',
  '2. From Symbol': 'EUR',
  '3. To Symbol': 'USD',
  '4. Last Refreshed': '2022-01-05 17:15:00',
  '5. Interval': '5min',
  '6. Output Size': 'Compact',
  '7. Time Zone': 'UTC'},
 'Time Series FX (5min)': {'2022-01-05 17:15:00': {'1. open': '1.13372',
   '2. high': '1.13378',
   '3. low': '1.13350',
   '4. close': '1.13350'},
  '2022-01-05 17:10:00': {'1. open': '1.13368',
   '2. high': '1.13392',
   '3. low': '1.13340',
   '4. close': '1.13375'},
  '2022-01-05 17:05:00': {'1. open': '1.13341',
   '2. high': '1.13375',
   '3. low': '1.13310',
   '4. close': '1.13350'},
  '2022-01-05 17:00:00': {'1. open': '1.13300',
   '2. high': '1.13347',
   '3. low': '1.13280',
   '4. close': '1.13342'},
  '2022-01-05 16:55:00': {'1. open': '1.13327',
   '2. high': '1.13334',
   '3. low': '1.13280',
   '4. close': '1.13298'},
  '2022-01-05 16:50:00': {'1. open': '1.13317',
   '2. high': '1.13327',
   '3. low': '1.13270',
   '4. clos

In [ ]:
data = {
    "function": "CURRENCY_EXCHANGE_RATE", # WEEKLY, MONTHLY possible
    "from_currency": "ETH",
    "to_currency": 'USD',
    "apikey": key
    }
r = requests.get(url, params=data)
data = r.json()
crypto_df = pd.DataFrame(data['Realtime Currency Exchange Rate'], index=[0]).T
crypto_df

<a id='7'></a>
## Cryptocurrencies

In [22]:
ticker = 'ETH'
url = 'https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol='+ticker+'&market=USD&interval=5min&apikey={key}'
r = requests.get(url)
data = r.json()
data

{'Meta Data': {'1. Information': 'Crypto Intraday (5min) Time Series',
  '2. Digital Currency Code': 'ETH',
  '3. Digital Currency Name': 'Ethereum',
  '4. Market Code': 'USD',
  '5. Market Name': 'United States Dollar',
  '6. Last Refreshed': '2022-01-11 15:55:00',
  '7. Interval': '5min',
  '8. Output Size': 'Compact',
  '9. Time Zone': 'UTC'},
 'Time Series Crypto (5min)': {'2022-01-11 15:55:00': {'1. open': '3123.57000',
   '2. high': '3130.76000',
   '3. low': '3123.41000',
   '4. close': '3125.55000',
   '5. volume': 331},
  '2022-01-11 15:50:00': {'1. open': '3116.36000',
   '2. high': '3126.63000',
   '3. low': '3113.44000',
   '4. close': '3123.61000',
   '5. volume': 599},
  '2022-01-11 15:45:00': {'1. open': '3112.14000',
   '2. high': '3121.72000',
   '3. low': '3110.75000',
   '4. close': '3116.36000',
   '5. volume': 887},
  '2022-01-11 15:40:00': {'1. open': '3124.93000',
   '2. high': '3125.21000',
   '3. low': '3110.82000',
   '4. close': '3112.30000',
   '5. volume': 

In [ ]:
data = {
    "function": "DIGITAL_CURRENCY_DAILY", # WEEKLY, MONTHLY possible
    "symbol": "ETH",
    "market": 'CNY',
    "apikey": key
    }
r = requests.get(url, params=data)
data = r.json()
crypto_df = pd.DataFrame(data['Time Series (Digital Currency Daily)']).T.reset_index()
crypto_df = crypto_df.rename(columns={"index": "Date"})
crypto_df['Date'] = pd.to_datetime(crypto_df['Date'])
crypto_df 

In [ ]:
data = {
    "function": "CURRENCY_EXCHANGE_RATE", # WEEKLY, MONTHLY possible
    "from_currency": "ETH",
    "to_currency": 'USD',
    "apikey": key
    }
r = requests.get(url, params=data)
data = r.json()
crypto_df = pd.DataFrame(data['Realtime Currency Exchange Rate'], index=[0]).T
crypto_df

<a id='8'></a>
## Mutual Funds

In [31]:
ticker = 'OMOIX'
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+ticker+'&apikey={key}'
r = requests.get(url)
data = r.json()
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'OMOIX',
  '3. Last Refreshed': '2022-01-03',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2022-01-03': {'1. open': '26.2200',
   '2. high': '26.2200',
   '3. low': '26.2200',
   '4. close': '26.2200',
   '5. volume': '0'},
  '2021-12-31': {'1. open': '26.1900',
   '2. high': '26.1900',
   '3. low': '26.1900',
   '4. close': '26.1900',
   '5. volume': '0'},
  '2021-12-30': {'1. open': '26.1600',
   '2. high': '26.1600',
   '3. low': '26.1600',
   '4. close': '26.1600',
   '5. volume': '0'},
  '2021-12-29': {'1. open': '26.1400',
   '2. high': '26.1400',
   '3. low': '26.1400',
   '4. close': '26.1400',
   '5. volume': '0'},
  '2021-12-28': {'1. open': '26.4600',
   '2. high': '26.4600',
   '3. low': '26.4600',
   '4. close': '26.4600',
   '5. volume': '0'},
  '2021-12-27': {'1. open': '26.4700',
   '2. high': '26.4700',
   '3. low': '26.4700'

<a id='9'></a>
## Treasury Yield

In [ ]:
treasury_yield = {
    "function": "TREASURY_YIELD",
    "interval": "weekly", # daily, monthly
    "maturity": "3month", # OPTIONAL 5year, 10year, 30year
    "apikey": key
}

<a id='10'></a>
## Stock Fundamentals

In [1]:
ticker = "IBM"
url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+ticker+'&apikey={key}'
r = requests.get(url)
data = r.json()

NameError: name 'requests' is not defined

<a id='11'></a>
## Financials

In [ ]:
document = 'INCOME_STATEMENT' # BALANCE_SHEET, CASH_FLOW
url = 'https://www.alphavantage.co/query?function='+document+'&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

<a id='12'></a>
## Realtime Data

In [15]:
def get_live_updates(symbol):
    api_key = key
    api_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    attributes = {'attributes':['symbol', 'open', 'high', 'low', 'price', 'volume', 'latest trading day', 'previous close', 'change', 'change percent']}
    attributes_df = pd.DataFrame(attributes)
    values = []
    for i in list(raw_df['Global Quote']):
        values.append(raw_df['Global Quote'][i])
    values_dict = {'values':values}
    values_df = pd.DataFrame(values).rename(columns = {0:'values'})
    frames = [attributes_df, values_df]
    df = pd.concat(frames, axis = 1, join = 'inner').set_index('attributes')
    return df

ibm_updates = get_live_updates('IBM')
ibm_updates

,values
attributes,
symbol,IBM
open,130.0000
high,130.9600
low,129.5200
price,130.6300
volume,3649044
latest trading day,2021-12-23
previous close,129.7500
change,0.8800


<a id='13'></a>
## Economic Indicators

In [91]:
gdp = {
    "function": "REAL_GDP",
    "interval": "annual", # quarterly
    "apikey": key
}

federal_funds_rate = {
    "function": "FEDERAL_FUNDS_RATE",
    "interval": "weekly", # daily, monthly
    "apikey": key
}
cpi = {
    "function": "CPI",
    "interval": "weekly", # daily, monthly
    "apikey": key
}
inflation = {
    "function": "INFLATION",
    "interval": "weekly", # daily, monthly
    "apikey": key
}
consumer_sentiment = {
    "function": "CONSUMER_SENTIMENT",
    "apikey": key
}
unemployment = {
    "function": "UNEMPLOYMENT",
    "apikey": key
}
r = requests.get(url, params=unemployment) # REPLACE 'params' with desired dict
data = r.json()
crypto_df = pd.DataFrame(data['data'])
crypto_df = crypto_df.set_index("date")
crypto_df

,value
date,
2021-11-01,4.2
2021-10-01,4.6
2021-09-01,4.8
2021-08-01,5.2
2021-07-01,5.4
...,...
1948-05-01,3.5
1948-04-01,3.9
1948-03-01,4.0


### Financials

In [ ]:
document = 'INCOME_STATEMENT' # BALANCE_SHEET, CASH_FLOW
url = 'https://www.alphavantage.co/query?function='+document+'&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

### Technical Indicators

In [101]:
# Lots to cover here, I've listed the most popular below
popular_ti = {
    "function": "ADX", # REPLACE: EMA, RSI, ADX, SMA
	"symbol": "IBM",
	"interval": "weekly",
	"time_period": "10",
	"series_type": "open",
    "apikey": key
}

r = requests.get(url, params=popular_ti)
data = r.json()
crypto_df = pd.DataFrame(data['Technical Analysis: ADX']).T # REPLACE to match the function
crypto_df

,ADX
2021-12-23,23.8256
2021-12-17,24.8406
2021-12-10,26.8894
2021-12-03,29.1061
2021-11-26,28.1847
...,...
2000-04-20,28.4723
2000-04-14,31.4184
2000-04-07,34.8993
2000-03-31,34.9256


## Extracting stock data (historical close price) for multiple stocks


In [ ]:
all_tickers = ["AAPL","MSFT","CSCO","AMZN","GOOG",
               "FB","BA","MMM","XOM","NKE","INTC"]
close_prices = pd.DataFrame()
api_call_count = 1
ts = TimeSeries(key=key, output_format='pandas')
start_time = time.time()
for ticker in all_tickers:
    data = ts.get_intraday(symbol=ticker,interval='1min', outputsize='compact')[0]
    api_call_count+=1
    data.columns = ["open","high","low","close","volume"]
    data = data.iloc[::-1]
    close_prices[ticker] = data["close"]
    if api_call_count==5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [ ]:
close_prices

## Extracting OHLCV data for multiple stocks


In [ ]:
all_tickers = ["AAPL","MSFT","CSCO","AMZN","GOOG",
               "FB","BA","MMM","XOM","NKE","INTC"]
ohlv_dict = {}
api_call_count = 1
ts = TimeSeries(key=open(key_path,'r').read(), output_format='pandas')
start_time = time.time()
for ticker in all_tickers:
    data = ts.get_intraday(symbol=ticker,interval='1min', outputsize='compact')[0]
    api_call_count+=1
    data.columns = ["open","high","low","close","volume"]
    data = data.iloc[::-1]
    ohlv_dict[ticker] = data
    if api_call_count==5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [ ]:
ohlv_dict['AMZN']